In [0]:
%run "./startup" 

In [0]:
params = WidgetParams({
    "sr_folio": str,    
    "sr_tipo_archivo": str,
    "sr_id_archivo": str,
    "sr_subproceso": str
})
# Validar widgets
params.validate()

In [0]:
conf = ConfManager()

#Archivos SQL
query = QueryManager()

#Conexion a base de datos
db = DBXConnectionManager()

In [0]:
#Query Extrae informacion 

statement = query.get_statement(
    "TRA_PATRIF_INCO_RECH_0020_TRANF_STA.sql",
    sr_folio = params.sr_folio,
    sr_tipo_archivo = params.sr_tipo_archivo,
    sr_id_archivo = params.sr_id_archivo,
    sr_subproceso = params.sr_subproceso,
)

db.write_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}", db.read_data("default", statement), "overwrite")

if conf.debug:
    display(db.read_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}"))

In [0]:
#Se limpia la tabla para asegurar que no se duplican por reproceso
statement = query.get_statement(
    "PATRIF_INCO_RECH_CIERREN_ETL.TTAFOTRAS_ETL_TRANSF_INFONAVIT_DELETE.sql",
    sr_folio=params.sr_folio,
    sr_id_archivo=params.sr_id_archivo,
    sr_tipo_archivo=params.sr_tipo_archivo
)
execution = db.execute_oci_dml(
    statement=statement, async_mode=False
)

In [0]:
#INSERTA EN LA TABLA

table_name = "CIERREN_ETL.TTAFOTRAS_ETL_TRANSF_INFONAVIT"

db.write_data(db.read_delta(f"DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}"), table_name, "default", "append")


In [0]:
#borrar la delta 
spark.sql(f"DROP TABLE IF EXISTS DELTA_TRA_LEE_ARCHIVO_{params.sr_folio}")